<a href="https://colab.research.google.com/github/Nastpavlova/PythonUrbanClassSample/blob/main/district_of_SPB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Самостоятельная работа**

In [ ]:
#установим необходимое для работы
!pip install folium -U
!pip install geopandas mapclassify osmnx

In [93]:
#чтобы pandas, geopandas, osmnx, traceback работали, подключим их: 
from traceback import print_exc
import pandas as pd
import geopandas as gpd
import osmnx as ox

In [94]:
def get_google_drive_download_url(url: str) -> str: #Функция возвращает ссылку для скачивания с google drive
    drive_id = url.split("/")[5]
    return f"https://drive.google.com/uc?export=download&id={drive_id}"

In [95]:
#выбрала Московский раойн, далее выгружу его из OpenStreetMap и визуализирую 
district = ox.geocode_to_gdf('Московский район, Санкт-Петербург') #датафрейм с геотмерией Московского района
district

,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((30.19877 59.80161, 30.19888 59.80152...",59.913051,59.744148,30.381655,30.198772,298093703,relation,338636,59.827803,30.31652,"Московский район, Saint Petersburg, Northweste...",boundary,administrative,0.600412


In [96]:
#визуализируем данные
district.explore(color='purple') #хочу фиолетовые

In [99]:
#Взяла второй файл из слоев с индустриальными предприятиями:
url = "https://drive.google.com/file/d/1W-j0pClTBLwULM7_OpjxLdNkh_AoeFJf/view?usp=sharing"

In [98]:
#данные по индустриальным предприятиям:
industr = gpd.read_file(get_google_drive_download_url(url)).to_crs(4326)
industr.head()

,Name,Organisation,Type,Category,Address,Coordinates y,Coordinates x,geometry
0,Канализационные очистные сооружения г. Кронштадт,Государственное унитарное предприятие «Водокан...,Водоканал,2,"улица Гидростроителей, 2\nКронштадт, Санкт-Пет...",60.014086,29.726834,MULTIPOINT (29.72683 60.01409)
1,Асфальто-бетонный завод,Санкт-Петербургское государственное унитарное ...,Другое,2,"Кронштадтское шоссе, 11\nКронштадт, Санкт-Пете...",60.007695,29.742914,MULTIPOINT (29.74291 60.00769)
2,Противотуберкулезный диспансер,Санкт-Петербургское государственное бюджетное ...,Медицинское учреждение,2,"проспект Обуховской Обороны, 231\nСанкт-Петерб...",59.864160,30.471406,MULTIPOINT (30.47141 59.86416)
3,СПБ ГБУЗ «Александровская больница»,Санкт-Петербургское государственное бюджетное ...,Медицинское учреждение,2,"проспект Солидарности, 4к3\nСанкт-Петербург, Р...",59.927472,30.489771,MULTIPOINT (30.48977 59.92747)
4,Санкт-Петербургская бумажная фабрика - филиал ...,Санкт-Петербургская бумажная фабрика - филиал ...,Другое,2,"набережная реки Фонтанки, 144\nСанкт-Петербург...",59.915514,30.289518,MULTIPOINT (30.28952 59.91551)


In [100]:
buffer_size = 150  #размер буфера в метрах

my_buffer = industr.to_crs(3857).buffer(buffer_size).to_crs(4326) #построение буфера
my_buffer.explore(color="purple") #тоже фиолетовый

In [90]:
#далее визуализируем Московский район СПБ, из которого исключены индустриальные предприятия вместе с их буфферами:
without_buffer = my_buffer.unary_union #атрибут unary_union объединит множество отдельных геометрий в единое целое
district.geometry.difference(without_buffer).explore(color='purple') #метод difference находит разность геометрий